# Unit 9: Grouping & Aggregation - Practical Applications

## 📚 Learning Objectives

By the end of this practical session, you will:

1. Master the split-apply-combine pattern
2. Use groupby() for data summarization
3. Apply multiple aggregation functions
4. Group by multiple columns
5. Create custom aggregation functions
6. Use transform, filter, and apply operations
7. Build pivot tables and cross-tabulations
8. Analyze real-world datasets with grouping

---

## 📦 Setup and Imports

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
pd.set_option('display.max_rows', 20)

print("✅ All libraries imported successfully!")
print(f"📊 NumPy version: {np.__version__}")
print(f"📈 Pandas version: {pd.__version__}")

---

## 📊 Part 1: Understanding the Split-Apply-Combine Pattern

### 1.1 Create Sample Sales Data

In [ ]:
# Create sample sales dataset
np.random.seed(42)

sales = pd.DataFrame({
    'Region': ['East', 'West', 'East', 'West', 'East', 'West', 'North', 'South'],
    'Product': ['A', 'B', 'A', 'B', 'C', 'A', 'B', 'C'],
    'Sales': [100, 200, 150, 180, 120, 220, 160, 140],
    'Costs': [60, 120, 90, 100, 70, 130, 95, 85],
    'Quantity': [10, 15, 12, 14, 11, 18, 13, 12]
})

print("Sales Dataset:")
print(sales)
print(f"\nShape: {sales.shape}")

### 1.2 Visualizing the Split-Apply-Combine Process

In [ ]:
print("="*70)
print("SPLIT-APPLY-COMBINE DEMONSTRATION")
print("="*70)

# STEP 1: SPLIT
print("\n1. SPLIT - Divide data by Region:")
print("-" * 40)
grouped = sales.groupby('Region')
for region, group_df in grouped:
    print(f"\n{region}:")
    print(group_df[['Region', 'Sales']])

# STEP 2: APPLY
print("\n2. APPLY - Calculate sum for each group:")
print("-" * 40)
for region, group_df in grouped:
    print(f"{region}: Sales sum = {group_df['Sales'].sum()}")

# STEP 3: COMBINE
print("\n3. COMBINE - Merge results:")
print("-" * 40)
result = grouped['Sales'].sum()
print(result)
print("="*70)

---

## 📊 Part 2: Basic GroupBy Operations

### 2.1 Simple Aggregations

In [ ]:
# Total sales by region
total_sales = sales.groupby('Region')['Sales'].sum()
print("Total Sales by Region:")
print(total_sales)
print()

# Average sales by region
avg_sales = sales.groupby('Region')['Sales'].mean()
print("Average Sales by Region:")
print(avg_sales)
print()

# Count transactions by region
count = sales.groupby('Region')['Sales'].count()
print("Transaction Count by Region:")
print(count)

### 2.2 Different Aggregation Functions

In [ ]:
# Demonstrate various aggregations
print("="*70)
print("VARIOUS AGGREGATION FUNCTIONS")
print("="*70)

grouped_region = sales.groupby('Region')['Sales']

print(f"Sum:     \n{grouped_region.sum()}\n")
print(f"Mean:    \n{grouped_region.mean()}\n")
print(f"Median:  \n{grouped_region.median()}\n")
print(f"Min:     \n{grouped_region.min()}\n")
print(f"Max:     \n{grouped_region.max()}\n")
print(f"Std Dev: \n{grouped_region.std()}\n")
print(f"Count:   \n{grouped_region.count()}\n")

### 2.3 Size vs Count

In [ ]:
# Create data with missing values
sales_with_nan = sales.copy()
sales_with_nan.loc[0, 'Sales'] = np.nan
sales_with_nan.loc[3, 'Sales'] = np.nan

print("Data with missing values:")
print(sales_with_nan[['Region', 'Sales']].head(5))
print()

# Compare size() and count()
print("size() - Counts ALL rows (including NaN):")
print(sales_with_nan.groupby('Region').size())
print()

print("count() - Counts only NON-NULL values:")
print(sales_with_nan.groupby('Region')['Sales'].count())
print()

print("💡 Key Difference:")
print("   size() counts all rows, count() counts only non-null values")

### 2.4 Reset Index

In [ ]:
# With index
with_index = sales.groupby('Region')['Sales'].sum()
print("With hierarchical index:")
print(with_index)
print(f"Type: {type(with_index)}")
print()

# Reset index
without_index = sales.groupby('Region')['Sales'].sum().reset_index()
print("After reset_index():")
print(without_index)
print(f"Type: {type(without_index)}")
print()

# With custom column name
named = sales.groupby('Region')['Sales'].sum().reset_index(name='Total_Sales')
print("With custom column name:")
print(named)

---

## 📊 Part 3: Multiple Aggregations

### 3.1 Using agg() with Multiple Functions

In [ ]:
# Multiple aggregations on one column
result = sales.groupby('Region')['Sales'].agg(['sum', 'mean', 'count', 'min', 'max', 'std'])

print("Multiple Aggregations on Sales:")
print(result)
print()
print("Column names:", result.columns.tolist())

### 3.2 Different Functions for Different Columns

In [ ]:
# Dictionary mapping different aggregations
agg_dict = {
    'Sales': ['sum', 'mean'],
    'Costs': ['sum', 'min', 'max'],
    'Quantity': 'count'
}

result = sales.groupby('Region').agg(agg_dict)
print("Different Aggregations for Each Column:")
print(result)
print()
print("Column structure (MultiIndex):")
print(result.columns.tolist())

### 3.3 Named Aggregations (Pandas 0.25+)

In [ ]:
# Named aggregations with clear column names
result = sales.groupby('Region').agg(
    total_sales=('Sales', 'sum'),
    avg_sales=('Sales', 'mean'),
    total_costs=('Costs', 'sum'),
    max_quantity=('Quantity', 'max'),
    num_transactions=('Sales', 'count')
).reset_index()

print("Named Aggregations (Clean Output):")
print(result)
print()
print("✅ Advantages:")
print("   • Clear, descriptive column names")
print("   • No MultiIndex to flatten")
print("   • Easy to understand and use")

### 3.4 Calculated Metrics

In [ ]:
# Calculate profit = sales - costs
result = sales.groupby('Region').agg(
    total_sales=('Sales', 'sum'),
    total_costs=('Costs', 'sum')
).reset_index()

# Add calculated column
result['profit'] = result['total_sales'] - result['total_costs']
result['profit_margin'] = (result['profit'] / result['total_sales'] * 100).round(2)

print("Regional Performance with Calculated Metrics:")
print(result)
print()

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

result.plot(x='Region', y=['total_sales', 'total_costs', 'profit'], 
            kind='bar', ax=axes[0])
axes[0].set_title('Sales, Costs, and Profit by Region', fontweight='bold')
axes[0].set_ylabel('Amount ($)')
axes[0].legend(title='Metric')

result.plot(x='Region', y='profit_margin', kind='bar', ax=axes[1], color='green')
axes[1].set_title('Profit Margin by Region', fontweight='bold')
axes[1].set_ylabel('Profit Margin (%)')
axes[1].axhline(y=result['profit_margin'].mean(), color='red', linestyle='--', 
                label='Average')
axes[1].legend()

plt.tight_layout()
plt.show()

---

## 📊 Part 4: Grouping by Multiple Columns

### 4.1 Basic Multi-Column Grouping

In [ ]:
# Group by Region and Product
multi_group = sales.groupby(['Region', 'Product'])['Sales'].sum()

print("Sales by Region and Product (MultiIndex):")
print(multi_group)
print()
print("Index levels:", multi_group.index.names)
print("Shape:", multi_group.shape)

### 4.2 Flattening MultiIndex Results

In [ ]:
# Reset index to flatten
flat_result = sales.groupby(['Region', 'Product'])['Sales'].sum().reset_index()

print("Flattened Result:")
print(flat_result)
print()
print("Now regular columns:", flat_result.columns.tolist())

### 4.3 Unstack for Pivot-like View

In [ ]:
# Unstack to create pivot-like matrix
pivot_view = sales.groupby(['Region', 'Product'])['Sales'].sum().unstack(fill_value=0)

print("Pivot-like View (unstack):")
print(pivot_view)
print()

# Visualize
pivot_view.plot(kind='bar', figsize=(10, 6))
plt.title('Sales by Region and Product', fontweight='bold', fontsize=14)
plt.xlabel('Region')
plt.ylabel('Sales ($)')
plt.legend(title='Product', bbox_to_anchor=(1.05, 1))
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 4.4 Complex Multi-Level Aggregations

In [ ]:
# Multiple aggregations with multiple groups
result = sales.groupby(['Region', 'Product']).agg(
    total_sales=('Sales', 'sum'),
    avg_sales=('Sales', 'mean'),
    total_quantity=('Quantity', 'sum'),
    transactions=('Sales', 'count')
).reset_index()

print("Comprehensive Analysis by Region and Product:")
print(result)
print()

# Find top combinations
top_5 = result.nlargest(5, 'total_sales')
print("Top 5 Region-Product Combinations:")
print(top_5[['Region', 'Product', 'total_sales']])

---

## 📊 Part 5: Real-World Dataset - Sales Analysis

### 5.1 Load and Explore Dataset

In [ ]:
# Create realistic sales dataset
np.random.seed(42)

# Generate dates
dates = pd.date_range('2024-01-01', periods=200, freq='D')

# Generate sales data
regions = np.random.choice(['East', 'West', 'North', 'South'], 200)
products = np.random.choice(['Product A', 'Product B', 'Product C', 'Product D'], 200)
categories = np.random.choice(['Electronics', 'Clothing', 'Food'], 200)
sales_amounts = np.random.uniform(50, 500, 200).round(2)
quantities = np.random.randint(1, 20, 200)
customers = np.random.choice([f'Customer_{i}' for i in range(1, 51)], 200)

sales_data = pd.DataFrame({
    'Date': dates,
    'Region': regions,
    'Product': products,
    'Category': categories,
    'Sales': sales_amounts,
    'Quantity': quantities,
    'Customer': customers
})

# Add costs (70-80% of sales)
sales_data['Costs'] = (sales_data['Sales'] * np.random.uniform(0.7, 0.8, 200)).round(2)

print("Sales Dataset Overview:")
print(sales_data.head(10))
print(f"\nShape: {sales_data.shape}")
print(f"Date range: {sales_data['Date'].min()} to {sales_data['Date'].max()}")
print(f"\nColumns: {sales_data.columns.tolist()}")

### 5.2 Sales by Region

In [ ]:
# Comprehensive regional analysis
regional_analysis = sales_data.groupby('Region').agg(
    total_revenue=('Sales', 'sum'),
    avg_transaction=('Sales', 'mean'),
    total_quantity=('Quantity', 'sum'),
    num_transactions=('Sales', 'count'),
    unique_customers=('Customer', 'nunique'),
    unique_products=('Product', 'nunique')
).reset_index()

# Calculate additional metrics
regional_analysis['avg_quantity_per_transaction'] = (
    regional_analysis['total_quantity'] / regional_analysis['num_transactions']
).round(2)

print("="*80)
print("REGIONAL PERFORMANCE ANALYSIS")
print("="*80)
print(regional_analysis.to_string(index=False))
print("="*80)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Total revenue
regional_analysis.plot(x='Region', y='total_revenue', kind='bar', 
                       ax=axes[0,0], color='steelblue')
axes[0,0].set_title('Total Revenue by Region', fontweight='bold')
axes[0,0].set_ylabel('Revenue ($)')

# Average transaction
regional_analysis.plot(x='Region', y='avg_transaction', kind='bar', 
                       ax=axes[0,1], color='coral')
axes[0,1].set_title('Average Transaction Value', fontweight='bold')
axes[0,1].set_ylabel('Amount ($)')

# Number of transactions
regional_analysis.plot(x='Region', y='num_transactions', kind='bar', 
                       ax=axes[1,0], color='green')
axes[1,0].set_title('Number of Transactions', fontweight='bold')
axes[1,0].set_ylabel('Count')

# Unique customers
regional_analysis.plot(x='Region', y='unique_customers', kind='bar', 
                       ax=axes[1,1], color='purple')
axes[1,1].set_title('Unique Customers', fontweight='bold')
axes[1,1].set_ylabel('Count')

plt.tight_layout()
plt.show()

### 5.3 Average Sales by Category

In [ ]:
# Category analysis
category_stats = sales_data.groupby('Category').agg(
    total_sales=('Sales', 'sum'),
    avg_sale=('Sales', 'mean'),
    median_sale=('Sales', 'median'),
    min_sale=('Sales', 'min'),
    max_sale=('Sales', 'max'),
    transactions=('Sales', 'count')
).reset_index()

print("Category Performance:")
print(category_stats.to_string(index=False))
print()

# Calculate market share
total_sales = category_stats['total_sales'].sum()
category_stats['market_share'] = (category_stats['total_sales'] / total_sales * 100).round(2)

print("Market Share by Category:")
print(category_stats[['Category', 'market_share']].to_string(index=False))

# Pie chart
plt.figure(figsize=(10, 6))
plt.pie(category_stats['total_sales'], labels=category_stats['Category'], 
        autopct='%1.1f%%', startangle=90, colors=['#ff9999','#66b3ff','#99ff99'])
plt.title('Sales Distribution by Category', fontweight='bold', fontsize=14)
plt.axis('equal')
plt.show()

### 5.4 Revenue by Category and Region

In [ ]:
# Two-way grouping: Category and Region
category_region = sales_data.groupby(['Category', 'Region'])['Sales'].sum().unstack(fill_value=0)

print("Sales by Category and Region:")
print(category_region)
print()

# Add totals
category_region['Total'] = category_region.sum(axis=1)
category_region.loc['Total'] = category_region.sum()

print("With Totals:")
print(category_region.round(2))
print()

# Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(category_region.iloc[:-1, :-1], annot=True, fmt='.0f', 
            cmap='YlOrRd', cbar_kws={'label': 'Sales ($)'})
plt.title('Sales Heatmap: Category vs Region', fontweight='bold', fontsize=14)
plt.xlabel('Region')
plt.ylabel('Category')
plt.tight_layout()
plt.show()

---

## 📊 Part 6: Transform Operations

### 6.1 Understanding Transform

In [ ]:
# Compare agg vs transform
sample = sales_data.head(10)[['Region', 'Sales']].copy()

print("Original Data:")
print(sample)
print(f"Shape: {sample.shape}")
print()

# Aggregate - reduces to one value per group
agg_result = sample.groupby('Region')['Sales'].mean()
print("Aggregate (mean):")
print(agg_result)
print(f"Shape: {agg_result.shape}")
print()

# Transform - returns same shape as input
transform_result = sample['Sales'] / sample.groupby('Region')['Sales'].transform('mean')
sample['sales_vs_regional_avg'] = transform_result
print("Transform (broadcast back to original):")
print(sample)
print(f"Shape: {sample.shape}")

### 6.2 Common Transform Use Cases

In [ ]:
# Create working copy
transform_df = sales_data.copy()

# 1. Percentage of group total
transform_df['pct_of_regional_sales'] = (
    transform_df['Sales'] / 
    transform_df.groupby('Region')['Sales'].transform('sum') * 100
).round(2)

# 2. Deviation from group mean
transform_df['deviation_from_regional_avg'] = (
    transform_df['Sales'] - 
    transform_df.groupby('Region')['Sales'].transform('mean')
).round(2)

# 3. Z-score within group (standardization)
transform_df['regional_zscore'] = transform_df.groupby('Region')['Sales'].transform(
    lambda x: (x - x.mean()) / x.std()
).round(2)

# 4. Rank within group
transform_df['regional_rank'] = transform_df.groupby('Region')['Sales'].rank(
    ascending=False, method='dense'
)

# Display results
print("Transform Operations Results:")
cols = ['Region', 'Sales', 'pct_of_regional_sales', 'deviation_from_regional_avg', 
        'regional_zscore', 'regional_rank']
print(transform_df[cols].head(15).to_string(index=False))
print()

# Find outliers (|z-score| > 2)
outliers = transform_df[abs(transform_df['regional_zscore']) > 2]
print(f"\nOutliers found (|z-score| > 2): {len(outliers)}")
if len(outliers) > 0:
    print(outliers[['Region', 'Sales', 'regional_zscore']].head())

---

## 📊 Part 7: Filter Operations

### 7.1 Filtering Groups

In [ ]:
# Filter: Keep only regions with total sales > 5000
high_sales_regions = sales_data.groupby('Region').filter(
    lambda x: x['Sales'].sum() > 5000
)

print("Original dataset:")
print(f"Regions: {sales_data['Region'].nunique()}")
print(f"Total rows: {len(sales_data)}")
print()

print("After filtering (regions with sales > $5000):")
print(f"Regions: {high_sales_regions['Region'].nunique()}")
print(f"Total rows: {len(high_sales_regions)}")
print()

print("Regions kept:")
print(high_sales_regions.groupby('Region')['Sales'].sum().sort_values(ascending=False))

### 7.2 Complex Filter Conditions

In [ ]:
# Multiple filter conditions
filtered = sales_data.groupby('Region').filter(
    lambda x: (x['Sales'].sum() > 5000) and (x['Sales'].mean() > 200)
)

print("Regions with total sales > $5000 AND average > $200:")
region_stats = filtered.groupby('Region').agg(
    total_sales=('Sales', 'sum'),
    avg_sales=('Sales', 'mean'),
    count=('Sales', 'count')
)
print(region_stats)
print()

# Filter by transaction count
active_customers = sales_data.groupby('Customer').filter(
    lambda x: len(x) >= 5  # At least 5 transactions
)

print(f"\nCustomers with at least 5 transactions:")
print(f"Active customers: {active_customers['Customer'].nunique()}")
print(f"Total transactions: {len(active_customers)}")

---

## 📊 Part 8: Custom Aggregation Functions

### 8.1 Lambda Functions

In [ ]:
# Custom aggregations with lambda
custom_aggs = sales_data.groupby('Region')['Sales'].agg([
    ('mean', 'mean'),
    ('range', lambda x: x.max() - x.min()),
    ('cv', lambda x: x.std() / x.mean() if x.mean() != 0 else 0),
    ('q75', lambda x: x.quantile(0.75))
])

print("Custom Aggregations:")
print(custom_aggs.round(2))
print()
print("Column explanations:")
print("  mean: Average sales")
print("  range: Difference between max and min")
print("  cv: Coefficient of variation (std/mean)")
print("  q75: 75th percentile")

### 8.2 Named Custom Functions

In [ ]:
# Define custom functions
def sales_range(x):
    """Calculate range of sales"""
    return x.max() - x.min()

def coefficient_of_variation(x):
    """Calculate CV (std/mean)"""
    return (x.std() / x.mean() * 100) if x.mean() != 0 else 0

def outlier_count(x):
    """Count outliers using IQR method"""
    q1, q3 = x.quantile(0.25), x.quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
    return ((x < lower) | (x > upper)).sum()

# Apply custom functions
result = sales_data.groupby('Region')['Sales'].agg([
    ('average', 'mean'),
    ('range', sales_range),
    ('cv_pct', coefficient_of_variation),
    ('outliers', outlier_count)
])

print("Custom Function Results:")
print(result.round(2))

### 8.3 Multi-Column Custom Aggregations

In [ ]:
# Custom function using multiple columns
def regional_summary(group):
    """Calculate comprehensive regional metrics"""
    return pd.Series({
        'total_revenue': group['Sales'].sum(),
        'total_costs': group['Costs'].sum(),
        'profit': group['Sales'].sum() - group['Costs'].sum(),
        'profit_margin_pct': ((group['Sales'].sum() - group['Costs'].sum()) / 
                              group['Sales'].sum() * 100) if group['Sales'].sum() > 0 else 0,
        'avg_transaction': group['Sales'].mean(),
        'transactions': len(group),
        'unique_products': group['Product'].nunique()
    })

# Apply custom summary function
regional_summary_df = sales_data.groupby('Region').apply(regional_summary).reset_index()

print("Comprehensive Regional Summary:")
print(regional_summary_df.round(2).to_string(index=False))
print()

# Find best performing region
best_region = regional_summary_df.loc[regional_summary_df['profit'].idxmax()]
print(f"\n🏆 Best Performing Region: {best_region['Region']}")
print(f"   Profit: ${best_region['profit']:.2f}")
print(f"   Profit Margin: {best_region['profit_margin_pct']:.2f}%")

---

## 📊 Part 9: Time-Based Grouping

### 9.1 Monthly Trends

In [ ]:
# Extract date components
sales_data['Year'] = sales_data['Date'].dt.year
sales_data['Month'] = sales_data['Date'].dt.month
sales_data['MonthName'] = sales_data['Date'].dt.month_name()
sales_data['DayOfWeek'] = sales_data['Date'].dt.day_name()

# Monthly aggregation
monthly_sales = sales_data.groupby(['Year', 'Month']).agg(
    total_sales=('Sales', 'sum'),
    avg_sales=('Sales', 'mean'),
    transactions=('Sales', 'count')
).reset_index()

print("Monthly Sales Trends:")
print(monthly_sales)
print()

# Visualize monthly trend
plt.figure(figsize=(12, 6))
plt.plot(range(len(monthly_sales)), monthly_sales['total_sales'], 
         marker='o', linewidth=2, markersize=8)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Sales ($)', fontsize=12)
plt.title('Monthly Sales Trend', fontweight='bold', fontsize=14)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 9.2 Day of Week Analysis

In [ ]:
# Sales by day of week
dow_sales = sales_data.groupby('DayOfWeek').agg(
    total_sales=('Sales', 'sum'),
    avg_sales=('Sales', 'mean'),
    transactions=('Sales', 'count')
).reset_index()

# Reorder days
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_sales['DayOfWeek'] = pd.Categorical(dow_sales['DayOfWeek'], categories=day_order, ordered=True)
dow_sales = dow_sales.sort_values('DayOfWeek')

print("Sales by Day of Week:")
print(dow_sales.to_string(index=False))
print()

# Find best day
best_day = dow_sales.loc[dow_sales['total_sales'].idxmax(), 'DayOfWeek']
print(f"🔥 Best sales day: {best_day}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

dow_sales.plot(x='DayOfWeek', y='total_sales', kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Total Sales by Day of Week', fontweight='bold')
axes[0].set_ylabel('Total Sales ($)')
axes[0].set_xlabel('Day')
axes[0].tick_params(axis='x', rotation=45)

dow_sales.plot(x='DayOfWeek', y='transactions', kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Transaction Count by Day of Week', fontweight='bold')
axes[1].set_ylabel('Number of Transactions')
axes[1].set_xlabel('Day')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---

## 📊 Part 10: Pivot Tables

### 10.1 Basic Pivot Table

In [ ]:
# Create pivot table
pivot = sales_data.pivot_table(
    values='Sales',
    index='Region',
    columns='Category',
    aggfunc='sum',
    fill_value=0
)

print("Pivot Table: Sales by Region and Category")
print(pivot.round(2))
print()

# Add margins (totals)
pivot_with_totals = sales_data.pivot_table(
    values='Sales',
    index='Region',
    columns='Category',
    aggfunc='sum',
    fill_value=0,
    margins=True,
    margins_name='Total'
)

print("With Row and Column Totals:")
print(pivot_with_totals.round(2))

### 10.2 Multiple Aggregations in Pivot

In [ ]:
# Multiple aggregation functions
multi_agg_pivot = sales_data.pivot_table(
    values='Sales',
    index='Region',
    columns='Category',
    aggfunc=['sum', 'mean', 'count'],
    fill_value=0
)

print("Pivot with Multiple Aggregations:")
print(multi_agg_pivot.round(2))

---

## 🎯 Summary: Key Takeaways

### What We Learned:

1. **Split-Apply-Combine Pattern:**
   - Split data into groups
   - Apply function to each group
   - Combine results

2. **Aggregation Methods:**
   - `sum()`, `mean()`, `median()`, `count()`
   - `min()`, `max()`, `std()`, `var()`
   - `agg()` for multiple functions

3. **Advanced Operations:**
   - `transform()` - Same shape as input
   - `filter()` - Select groups
   - `apply()` - Custom functions

4. **Multi-Level Grouping:**
   - Group by multiple columns
   - Create pivot tables
   - Hierarchical indexing

5. **Real-World Applications:**
   - Sales analysis by region
   - Time-based trends
   - Customer segmentation
   - Performance metrics

### Important Reminders:

✅ **Choose the right method:**
   - Simple aggregation → direct methods
   - Multiple aggregations → `agg()`
   - Same-shape output → `transform()`
   - Group filtering → `filter()`
   - Complex logic → `apply()`

✅ **Handle missing values explicitly**  
✅ **Reset index when needed**  
✅ **Use built-in functions (faster)**  
✅ **Name aggregated columns clearly**  
✅ **Validate results**  

---

**🎉 Congratulations!** You've completed the practical session on Grouping & Aggregation!

**Next Steps:**
1. Practice with the exercises
2. Apply to your own datasets
3. Explore advanced topics (window functions, custom aggregators)
4. Learn performance optimization techniques

**Remember:** Grouping and aggregation are fundamental to data analysis. Master these techniques to unlock powerful insights from your data!